In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, differential_evolution, basinhopping
from typing import Callable, Dict, Tuple, Optional, Union
import warnings
warnings.filterwarnings('ignore')

class RobustQLEVolatilityModel:
    
    def __init__(self, alpha_loss: float = None, c: float = None):
        self.alpha_loss = alpha_loss
        self.c = c
        self.params = None
        self.param_names = ['omega', 'gamma', 'beta']
        if alpha_loss is None:
            self.param_names.append('alpha_loss')
        if c is None:
            self.param_names.append('c')
        self.fitted_volatility = None
        self.residuals = None
    
    # [Keep all your existing methods: _rho_derivative, _rho_second_derivative, etc.]
    # I'm only showing the modified fit method for brevity
    import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt
from typing import Callable, Dict, Tuple, Optional, Union
import autograd 
from autograd import grad, jacobian
import warnings
warnings.filterwarnings('ignore')

class RobustQLEVolatilityModel:
    
    
    def __init__(self, alpha_loss: float = None, c: float = None):
        
        self.alpha_loss = alpha_loss
        self.c = c
        self.params = None
        self.param_names = ['omega', 'gamma', 'beta']
        if alpha_loss is None:
            self.param_names.append('alpha_loss')
        if c is None:
            self.param_names.append('c')
        self.fitted_volatility = None
        self.residuals = None
    
    def _rho_derivative(self, e: np.ndarray, alpha: float, c: float) -> np.ndarray:
        
        # Handle scalar inputs by converting to numpy arrays
        if np.isscalar(e):
            e = np.array([e])
            scalar_input = True
        else:
            scalar_input = False
            
        if alpha == 2:
            # L2 loss (least squares)
            result = e / (c**2)
        elif alpha == 0:
            # Cauchy/Lorentzian loss
            result = (2 * e) / (e**2 + 2 * c**2)
        elif alpha == float('-inf'):
            # Welsch/Leclerc loss
            result = (e / c**2) * np.exp(-0.5 * (e/c)**2)
        else:
            # General case
            result = (e / c**2) * np.power((e**2 / c**2) / np.abs(alpha-2) + 1, alpha/2 - 1)
        
        # Return scalar if input was scalar
        if scalar_input:
            return result[0]
        return result
    
    #def _rho_second_derivative(self, e: np.ndarray, alpha: float, c: float) -> np.ndarray:
        
        # Handle scalar inputs by converting to numpy arrays
        if np.isscalar(e):
            e = np.array([e])
            scalar_input = True
        else:
            scalar_input = False
            
        if alpha == 2:
            # L2 loss (least squares)
            result = np.ones_like(e) / (c**2)
        elif alpha == 0:
            # Cauchy/Lorentzian loss
            denom = (1 + 0.5 * (e/c)**2)**2
            result = (1 / c**2) * (1 - 0.5 * (e/c)**2) / denom
        elif alpha == float('-inf'):
            # Welsch/Leclerc loss
            result = (1 / c**2) * (1 - (e**2 / c**2)) * np.exp(-0.5 * (e/c)**2)
        else:
            # General case
            base_term = np.power((e/c)**2 / np.abs(alpha-2) + 1, alpha/2 - 2)
            part1 = (1 / c**2) * base_term
            part2 = (e/c)**2 / np.abs(alpha-2) + 1
            part3 = part2 + (alpha/2 - 1) * (2 * e**2) / (c**2 * np.abs(alpha-2))
            result = part1 * part3
            
        
        # Return scalar if input was scalar
        if scalar_input:
            return result[0]
        return result
    def _rho_second_derivative(self, e: np.ndarray, alpha: float, c: float) -> np.ndarray:
        
        # Handle scalar inputs by converting to numpy arrays
        if np.isscalar(e):
            e = np.array([e])
            scalar_input = True
        else:
            scalar_input = False
            
        if alpha == 2:
            # L2 loss (least squares)
            result = -np.ones_like(e) / (c**2)
        elif alpha == 0:
            # Cauchy/Lorentzian loss
            denom = (e**2 + 2 * c**2)**2
            result = -(2*(e**2 + 2* c**2) - 4 * e **2) / denom
        elif alpha == float('-inf'):
            # Welsch/Leclerc loss
            first_term = ((e)**2 * np.exp( -((e**2)/(2* (c**2)))))/(c**4)
            second_term = np.exp( -((e**2)/(2* (c**2))))/(c**2)
            result = (first_term - second_term)
        else:
            # General case
            e2 = (e)**2
            denom = c**2 * np.abs(alpha - 2)
            A = e2 / denom + 1

            term1 = -2 * (alpha/2 - 1) * A**(alpha/2 - 2) * e2
            term1 /= (c**4 * np.abs(alpha - 2))

            term2 = - A**(alpha/2 - 1) / (c**2)

            return term1 + term2
            
        
        # Return scalar if input was scalar
        if scalar_input:
            return result[0]
        return result
    
    #def dpsi_dalpha(self, e_t, c, alpha_loss):
        
        # define a small neighborhood around the problematic points:
        eps = 1e-4

        # if α is within ±eps of 2, treat it as exactly 2 (L2 case)
        if abs(alpha_loss - 2) < eps:
            delta = 1e-5
            psi_plus = self._rho_derivative(e_t, alpha_loss + delta, self.c) 
            psi_minus = self._rho_derivative(e_t, alpha_loss - delta, self.c) 
            dpsi_dalpha = (psi_plus - psi_minus) / (2 * delta)
            return dpsi_dalpha  # ∂ψ/∂α = 0 for the L2 loss

        # similarly guard α≈0
        if abs(alpha_loss - 0) < eps:
            # use the closed-form ∂ψ/∂α at α=0 (which you've already set to 0)
            delta = 1e-5
            psi_plus = self._rho_derivative(e_t, alpha_loss + delta, self.c) 
            psi_minus = self._rho_derivative(e_t, alpha_loss - delta, self.c)
            dpsi_dalpha = (psi_plus - psi_minus) / (2 * delta)
            return dpsi_dalpha

        # if you ever parameterize "α = −∞" as, say, alpha_loss < some negative cutoff
        if alpha_loss < -1e3:
            delta = 1e-5
            psi_plus = self._rho_derivative(e_t, alpha_loss + delta, self.c) 
            psi_minus = self._rho_derivative(e_t, alpha_loss - delta, self.c) 
            dpsi_dalpha = (psi_plus - psi_minus) / (2 * delta)
            return dpsi_dalpha  

        # otherwise you're safely away from the singularities, so use your general formula:
        abs_alpha_minus_2 = np.abs(alpha_loss - 2)
        denominator_inner = c**2 * abs_alpha_minus_2
        inner_term = (e_t**2) / denominator_inner + 1
        power_term = inner_term**(alpha_loss / 2 - 1)

        log_term = np.log(inner_term) / 2
        frac_term = (e_t**2 * (alpha_loss / 2 - 1)) / (
            c**2 * inner_term * abs_alpha_minus_2 * (alpha_loss - 2)
        )

        numerator = e_t * power_term * (log_term - frac_term)

        return (numerator / c**2)
    def dpsi_dalpha(self, e_t, c, alpha_loss):
        
        # define a small neighborhood around the problematic points:
        eps = 1e-4

        # if α is within ±eps of 2, treat it as exactly 2 (L2 case)
        if abs(alpha_loss - 2) < eps:
            delta = 1e-5
            psi_plus = self._rho_derivative(e_t, alpha_loss + delta, self.c) 
            psi_minus = self._rho_derivative(e_t, alpha_loss - delta, self.c) 
            dpsi_dalpha = (psi_plus - psi_minus) / (2 * delta)
            return dpsi_dalpha  # ∂ψ/∂α = 0 for the L2 loss

        # similarly guard α≈0
        if abs(alpha_loss - 0) < eps:
            # use the closed-form ∂ψ/∂α at α=0 (which you've already set to 0)
            delta = 1e-5
            psi_plus = self._rho_derivative(e_t, alpha_loss + delta, self.c) 
            psi_minus = self._rho_derivative(e_t, alpha_loss - delta, self.c) 
            dpsi_dalpha = (psi_plus - psi_minus) / (2 * delta)
            return dpsi_dalpha

        # if you ever parameterize "α = −∞" as, say, alpha_loss < some negative cutoff
        if alpha_loss < -1e3:
            delta = 1e-5
            psi_plus = self._rho_derivative(e_t, alpha_loss + delta, self.c) 
            psi_minus = self._rho_derivative(e_t, alpha_loss - delta, self.c)
            dpsi_dalpha = (psi_plus - psi_minus) / (2 * delta)
            return dpsi_dalpha  

        # otherwise you're safely away from the singularities, so use your general formula:
        alpha= alpha_loss
        e2 = e_t**2
        denom = c**2 * np.abs(alpha - 2)
        A = (e2 / denom) + 1

        # Compute the bracket term
        term_log = np.log(A) / 2
        term_frac = (e2 * (alpha/2 - 1)) / (c**2 * A * np.abs(alpha - 2) * (alpha - 2))
        bracket = term_log - term_frac

        # Combine everything
        result = (e_t * A**(alpha/2 - 1) * bracket) / (c**2)
        return result

    
    #def dpsi_dc(self, e_t, c, alpha_loss):
       
        eps = 1e-4

        if alpha_loss == 2:
            # dψ/dc = -2e / c³
            return -2 * e_t / c**3

        elif abs(alpha_loss) < eps:
            # dψ/dc = -8ec / (e² + 2c²)²
            return -8 * e_t * c / (e_t**2 + 2 * c**2)**2

        elif alpha_loss < -1e3:
            # dψ/dc = ( -2e / c³ + e³ / c⁵ ) * exp(-½ (e/c)²)
            z = e_t / c
            exp_term = np.exp(-0.5 * z**2)
            return (-2 * e_t / c**3 + e_t**3 / c**5) * exp_term

        else:
            # General case:
            # dψ/dc = [ -2e / c³ 
            #           + (e / c²)(α/2 - 1)( (e² / (c²|α−2|) + 1 )**(α/2 - 2) ) * (-2e² / (c³|α−2|)) ]
            #         * (e² / (c²|α−2|) + 1 )**(α/2 - 1)
            abs_alpha_diff = np.abs(alpha_loss - 2)
            z_sq = (e_t**2) / (c**2 * abs_alpha_diff)
            base = z_sq + 1
            power1 = (alpha_loss / 2) - 2
            power2 = (alpha_loss / 2) - 1

            first_term = -2 * e_t / c**3
            second_term = (e_t / c**2) * ((alpha_loss / 2) - 1) * (base**power1) * (-2 * e_t**2 / (c**3 * abs_alpha_diff))

            return (first_term + second_term) * base**power2
    def dpsi_dc(self, e_t, c, alpha_loss):
       
        eps = 1e-4

        if alpha_loss == 2:
            # dψ/dc = -2e / c³
            return -2 * e_t / c**3

        elif abs(alpha_loss) < eps:
            # dψ/dc = -8ec / (e² + 2c²)²
            return -8 * e_t * c / (e_t**2 + 2 * c**2)**2

        elif alpha_loss < -1e3:
            # dψ/dc = ( -2e / c³ + e³ / c⁵ ) * exp(-½ (e/c)²)
            z = e_t / c
            exp_term = np.exp(-0.5 * z**2)
            return (-2 * e_t / c**3 + e_t**3 / c**5) * exp_term

        else:
            alpha = alpha_loss
            e2 = e_t**2
            denom = np.abs(alpha - 2) * c**2
            A = e2 / denom + 1

            term1 = -2 * e_t * A**(alpha/2 - 1) / (c**3)
            term2 = -2 * e_t**3 * (alpha/2 - 1) * A**(alpha/2 - 2) / (np.abs(alpha - 2) * c**5)
            
            return term1 + term2
            # General case:
            
    
    def _filter_volatility(self, y: np.ndarray, params: np.ndarray) -> np.ndarray:
        
        T = len(y)
        omega, gamma, beta = params[:3]
        
        param_idx = 3
        
        # Get alpha_loss parameter
        if self.alpha_loss is None:
            alpha_loss = params[param_idx]
            param_idx += 1
        else:
            alpha_loss = self.alpha_loss
        
        # Get c parameter
        if self.c is None:
            c = params[param_idx]
        else:
            c = self.c
        
        # Initialize volatility with the unconditional variance
        f = np.zeros(T+1)
        f[0] = omega/(1-beta)
        
        # Recursively update the volatility
        for t in range(T):
            e_t = y[t]**2 - f[t]
            psi_t = self._rho_derivative(e_t, alpha_loss, c)   
            f[t+1] = omega + gamma * psi_t + beta * f[t]
            
            # Enforce positive volatility
            f[t+1] = max(f[t+1], 1e-12)
        
        return f[1:]
    
    def _compute_derivatives(self, y: np.ndarray, f: np.ndarray, params: np.ndarray) -> np.ndarray:
       
        T = len(y)
        omega, gamma, beta = params[:3]
        
        param_idx = 3
        n_params = 3
        
        # Get alpha_loss parameter
        if self.alpha_loss is None:
            alpha_loss = params[param_idx]
            param_idx += 1
            n_params += 1
        else:
            alpha_loss = self.alpha_loss
        
        # Get c parameter
        if self.c is None:
            c = params[param_idx]
            n_params += 1
        else:
            c = self.c
        
        # Initialize derivatives of f_t with respect to theta
        e0 = y[0]**2 - f[0]
        psi0 =  self._rho_derivative(e0, alpha_loss, c)
        d2psi0_df = self._rho_second_derivative(e0, alpha_loss, c) 
        df_dtheta = np.zeros((T, n_params))
        df_dtheta[0, 0] = 1                  # ∂f₁/∂ω
        df_dtheta[0, 1] = psi0               # ∂f₁/∂γ
        df_dtheta[0, 2] = f[0]               # ∂f₁/∂β
        
        # Initialize derivatives for alpha_loss and c if they are estimated
        param_idx = 3
        
        if self.alpha_loss is None:
            dpsi0_dalpha = self.dpsi_dalpha(e0, c, alpha_loss)
            df_dtheta[0, param_idx] = gamma * dpsi0_dalpha  # ∂f₁/∂α
            param_idx += 1
        
        if self.c is None:
            dpsi0_dc = self.dpsi_dc(e0, c, alpha_loss)
            df_dtheta[0, param_idx] = gamma * dpsi0_dc  # ∂f₁/∂c
    
        # According to the recursive formula:
        
        for t in range(1, T):
            e_t = (y[t-1]**2 - f[t-1])
            psi_t = self._rho_derivative(e_t, alpha_loss, c)  
            d2psi_df = self._rho_second_derivative(e_t, alpha_loss, c)   
            
            # Common term in recursive updates
            common_term = gamma * d2psi_df + beta
            
            # Derivative with respect to parameters
            # For omega (∂ω/∂θ_0 = 1, else 0)
            df_dtheta[t, 0] = 1 + df_dtheta[t-1, 0] * common_term
            
            # For gamma (∂γ/∂θ_1 = 1, else 0)
            df_dtheta[t, 1] = psi_t + df_dtheta[t-1, 1] * common_term
            
            # For beta (∂β/∂θ_2 = 1, else 0)
            df_dtheta[t, 2] = f[t-1] + df_dtheta[t-1, 2] * common_term
            
            # Reset param_idx for additional parameters
            param_idx = 3
            
            # Derivative with respect to alpha_loss, if applicable
            if self.alpha_loss is None:
                dpsi_dalpha = self.dpsi_dalpha(e_t, c, alpha_loss)
                df_dtheta[t, param_idx] = gamma * dpsi_dalpha + common_term * df_dtheta[t-1, param_idx]
                param_idx += 1
            
            # Derivative with respect to c, if applicable
            if self.c is None:
                dpsi_dc = self.dpsi_dc(e_t, c, alpha_loss)
                df_dtheta[t, param_idx] = gamma * dpsi_dc + common_term * df_dtheta[t-1, param_idx]
        
        return df_dtheta
    
    def _qle_objective(self, params: np.ndarray, y: np.ndarray) -> float:
        
        try:
            # Apply parameter constraints
            param_idx = 3
            
            if self.alpha_loss is None:
                # Ensure alpha_loss is in reasonable range
                if params[param_idx] < -10 or params[param_idx] > 10:
                    return 1e10
                param_idx += 1
            
            if self.c is None:
                # Ensure c is positive and reasonable
                if params[param_idx] <= 0 or params[param_idx] > 10:
                    return 1e10
            
            # Basic parameter constraints for volatility model stability
            if params[0] <= 0 or params[1] < 0 or params[2] < 0 or params[2] >= 1 or params[1] + params[2] >= 0.999:
                return 1e10
            
            # Filter volatility
            f = self._filter_volatility(y, params)
            
            # Compute residuals - h_t is defined as y_t^k - f_t(θ)
            h_t = y**2 - f
            
            # Use f as the conditional variance approximation
            sigma2_t = f
            
            # Compute derivatives of f_t with respect to parameters
            df_dtheta = self._compute_derivatives(y, f, params)
            
            # G_t(θ) = (1/T) * sum[ h_t(θ) / σ²_t(θ) * ∂f_t(θ)/∂θ ]
            G_t = np.sum(h_t.reshape(-1, 1) / sigma2_t.reshape(-1, 1) * df_dtheta, axis=0) / len(y)
            
            # The objective is to minimize ||G_t(θ)||²
            obj = np.linalg.norm(G_t)
            
            return obj
        except Exception as e:
            print(f"Error in objective function: {e}")
            return 1e10
    
    def fit(self, y: np.ndarray, initial_params: Optional[Dict] = None, 
            method: str = 'Nelder-Mead', maxiter: int = 2000) -> Dict:
        
        # Set default initial parameters if not provided
        if initial_params is None:
            initial_params = {
                'omega': 0.07, 
                'gamma': 0.11, 
                'beta': 0.8
            }
            if self.alpha_loss is None:
                initial_params['alpha_loss'] = 1  # Default value close to Cauchy loss
            
            if self.c is None:
                initial_params['c'] = 1.2  # Default value for scale parameter
        
        # Prepare initial parameter array
        init_params = np.array([initial_params[name] for name in self.param_names])
        
        # Run optimization
        options = {'maxiter': maxiter, 'disp': True}
        
        # Different optimization methods may work better in different cases
        if method == 'Nelder-Mead':
            options['adaptive'] = True  # Use adaptive Nelder-Mead for better convergence
            result = minimize(
                self._qle_objective, 
                init_params, 
                args=(y,), 
                method=method, 
                options=options,
            )
        elif method == 'BFGS':
            options = {'maxiter': maxiter, 'gtol': 1e-6}
            result = minimize(
                self._qle_objective, 
                init_params, 
                args=(y,), 
                method=method, 
                options=options,
            )
        elif method == 'differential_evolution':
            # Set up bounds for differential evolution
            bounds = []
            
            # Basic parameters bounds
            bounds.extend([(0.001, 0.5), (0.001, 0.5), (0.6, 0.999)])
            
            # Alpha bounds if estimated
            if self.alpha_loss is None:
                bounds.append((-5, 5))
            
            # c bounds if estimated
            if self.c is None:
                bounds.append((0.1, 5.0))
            
            result = differential_evolution(
                self._qle_objective,
                bounds=bounds,
                args=(y,),
                strategy='best1bin',
                maxiter=maxiter,
                disp=True,
                polish=True
            )
        
        if not result.success and method != 'differential_evolution':
            print(f"Warning: Optimization did not converge: {result.message}")
            
            # Try again with different method if first one failed
            if method == 'Nelder-Mead':
                print("Trying BFGS method instead...")
                result = minimize(
                    self._qle_objective,
                    init_params,
                    args=(y,),
                    method='BFGS',
                    options={'maxiter': maxiter}
                )
        
        # Store parameters
        self.params = {name: val for name, val in zip(self.param_names, result.x)}
        
        # Compute fitted volatility
        param_array = np.array([self.params[name] for name in self.param_names])
        self.fitted_volatility = self._filter_volatility(y, param_array)
        self.residuals = y**2 - self.fitted_volatility
        
        print(f"Optimization result: {result.message}")
        print(f"Parameters: {self.params}")
        
        return self.params    
    def _parameter_bounds_check(self, params: np.ndarray) -> bool:
        """Check if parameters are within valid bounds"""
        param_idx = 3
        
        # Basic parameter constraints
        if params[0] <= 0 or params[1] < 0 or params[2] < 0 or params[2] >= 1:
            return False
        if params[1] + params[2] >= 0.999:
            return False
            
        if self.alpha_loss is None:
            if params[param_idx] < -10 or params[param_idx] > 10:
                return False
            param_idx += 1
        
        if self.c is None:
            if params[param_idx] <= 0 or params[param_idx] > 10:
                return False
                
        return True
    
    def _generate_random_start(self, bounds: list = None) -> np.ndarray:
        """Generate random starting parameters within bounds"""
        if bounds is None:
            # Default bounds
            omega_bound = (0.001, 0.2)
            gamma_bound = (0.01, 0.5)
            beta_bound = (0.1, 0.95)
            
            params = [
                np.random.uniform(*omega_bound),
                np.random.uniform(*gamma_bound),
                np.random.uniform(*beta_bound)
            ]
            
            if self.alpha_loss is None:
                params.append(np.random.uniform(-3, 3))
            
            if self.c is None:
                params.append(np.random.uniform(0.5, 3.0))
                
        else:
            params = [np.random.uniform(low, high) for low, high in bounds]
        
        return np.array(params)
    
    def _multi_start_optimization(self, y: np.ndarray, n_starts: int = 10, 
                                method: str = 'L-BFGS-B', maxiter: int = 1000) -> Dict:
        """Run optimization from multiple random starting points"""
        
        # Set up bounds for constrained optimization
        bounds = []
        bounds.extend([(0.001, 0.5), (0.001, 0.5), (0.1, 0.999)])
        
        if self.alpha_loss is None:
            bounds.append((-5, 5))
        
        if self.c is None:
            bounds.append((0.1, 5.0))
        
        best_result = None
        best_obj = float('inf')
        all_results = []
        
        print(f"Running {n_starts} optimizations from different starting points...")
        
        for i in range(n_starts):
            try:
                # Generate random starting point
                init_params = self._generate_random_start(bounds)
                
                # Run optimization
                result = minimize(
                    self._qle_objective,
                    init_params,
                    args=(y,),
                    method=method,
                    bounds=bounds,
                    options={'maxiter': maxiter, 'disp': False}
                )
                
                all_results.append(result)
                
                if result.fun < best_obj:
                    best_obj = result.fun
                    best_result = result
                    
                print(f"  Start {i+1}: obj={result.fun:.6f}, success={result.success}")
                
            except Exception as e:
                print(f"  Start {i+1}: Failed with error {e}")
                continue
        
        return best_result, all_results
    
    def fit(self, y: np.ndarray, initial_params: Optional[Dict] = None, 
            method: str = 'basin_hopping', maxiter: int = 2000, 
            n_starts: int = 5, **kwargs) -> Dict:
        """
        Fit the model with various optimization strategies
        
        Parameters:
        -----------
        method : str
            'basin_hopping', 'multi_start', 'differential_evolution', 'Nelder-Mead', 'L-BFGS-B'
        n_starts : int
            Number of starting points for multi_start method
        **kwargs : additional arguments for specific methods
        """
        
        # Set default initial parameters
        if initial_params is None:
            initial_params = {
                'omega': 0.07, 
                'gamma': 0.11, 
                'beta': 0.8
            }
            if self.alpha_loss is None:
                initial_params['alpha_loss'] = 1.0
            if self.c is None:
                initial_params['c'] = 1.2
        
        init_params = np.array([initial_params[name] for name in self.param_names])
        
        # Set up bounds for constrained methods
        bounds = []
        bounds.extend([(0.001, 0.5), (0.001, 0.5), (0.1, 0.999)])
        
        if self.alpha_loss is None:
            bounds.append((-5, 5))
        
        if self.c is None:
            bounds.append((0.1, 5.0))
        
        print(f"Fitting model using {method} method...")
        
        if method == 'basin_hopping':
            # Basin-hopping with custom step-taking and acceptance criteria
            
            class BoundedStep:
                def __init__(self, bounds, stepsize=0.1):
                    self.bounds = np.array(bounds)
                    self.stepsize = stepsize
                
                def __call__(self, x):
                    """Take a random step within bounds"""
                    # Generate random perturbation
                    perturbation = np.random.uniform(-self.stepsize, self.stepsize, len(x))
                    
                    # Apply perturbation
                    x_new = x + perturbation
                    
                    # Clip to bounds
                    x_new = np.clip(x_new, self.bounds[:, 0], self.bounds[:, 1])
                    
                    return x_new
            
            def accept_test(f_new, x_new, f_old, x_old):
                """Custom acceptance criterion"""
                # Always accept if better
                if f_new < f_old:
                    return True
                # Accept with probability based on improvement
                delta = f_new - f_old
                prob = np.exp(-delta * 10)  # Adjust temperature
                return np.random.random() < prob
            
            # Create step-taking class
            step_class = BoundedStep(bounds, stepsize=kwargs.get('stepsize', 0.1))
            
            # Run basin-hopping
            result = basinhopping(
                self._qle_objective,
                init_params,
                minimizer_kwargs={
                    'method': 'L-BFGS-B',
                    'bounds': bounds,
                    'args': (y,),
                    'options': {'maxiter': 200}
                },
                niter=kwargs.get('niter', 100),
                T=kwargs.get('T', 1.0),
                stepsize=kwargs.get('stepsize', 0.1),
                take_step=step_class,
                accept_test=accept_test,
                disp=True
            )
            
        elif method == 'multi_start':
            result, all_results = self._multi_start_optimization(
                y, n_starts=n_starts, maxiter=maxiter
            )
            
        elif method == 'differential_evolution':
            result = differential_evolution(
                self._qle_objective,
                bounds=bounds,
                args=(y,),
                strategy='best1bin',
                maxiter=maxiter,
                disp=True,
                polish=True,
                **kwargs
            )
            
        elif method in ['L-BFGS-B', 'SLSQP']:
            result = minimize(
                self._qle_objective,
                init_params,
                args=(y,),
                method=method,
                bounds=bounds,
                options={'maxiter': maxiter, 'disp': True}
            )
            
        else:  # Nelder-Mead or other unconstrained methods
            options = {'maxiter': maxiter, 'disp': True}
            if method == 'Nelder-Mead':
                options['adaptive'] = True
                
            result = minimize(
                self._qle_objective,
                init_params,
                args=(y,),
                method=method,
                options=options
            )
        
        # Handle failed optimization
        if not result.success and method not in ['differential_evolution', 'basin_hopping']:
            print(f"Warning: {method} optimization failed: {result.message}")
            print("Trying basin-hopping as fallback...")
            
            # Fallback to basin-hopping
            step_class = BoundedStep(bounds, stepsize=0.05)
            result = basinhopping(
                self._qle_objective,
                init_params,
                minimizer_kwargs={
                    'method': 'L-BFGS-B',
                    'bounds': bounds,
                    'args': (y,),
                    'options': {'maxiter': 200}
                },
                niter=50,
                take_step=step_class,
                disp=True
            )
        
        # Store results
        self.params = {name: val for name, val in zip(self.param_names, result.x)}
        
        # Compute fitted volatility
        param_array = np.array([self.params[name] for name in self.param_names])
        self.fitted_volatility = self._filter_volatility(y, param_array)
        self.residuals = y**2 - self.fitted_volatility
        
        print(f"Optimization result: {result.message if hasattr(result, 'message') else 'Completed'}")
        print(f"Final objective value: {result.fun:.8f}")
        print(f"Parameters: {self.params}")
        
        return self.params
    
    def compare_methods(self, y: np.ndarray, methods: list = None, **kwargs) -> Dict:
        """
        Compare different optimization methods
        
        Returns dictionary with results from each method
        """
        if methods is None:
            methods = ['L-BFGS-B', 'basin_hopping', 'differential_evolution', 'multi_start']
        
        results = {}
        
        for method in methods:
            print(f"\n{'='*50}")
            print(f"Testing method: {method}")
            print(f"{'='*50}")
            
            try:
                # Reset parameters
                self.params = None
                self.fitted_volatility = None
                self.residuals = None
                
                # Fit model
                params = self.fit(y, method=method, **kwargs)
                
                # Store results
                results[method] = {
                    'params': params.copy(),
                    'objective': self._qle_objective(
                        np.array([params[name] for name in self.param_names]), y
                    ),
                    'success': True
                }
                
            except Exception as e:
                print(f"Method {method} failed: {e}")
                results[method] = {
                    'params': None,
                    'objective': float('inf'),
                    'success': False,
                    'error': str(e)
                }
    
        return results
    def plot_volatility(self, y: np.ndarray, true_vol: np.ndarray = None, title: str = "Estimated Volatility") -> None:
        
        if self.fitted_volatility is None:
            raise ValueError("Model must be fit before plotting")
        
        plt.figure(figsize=(12, 8))
        
        # Plot original data
        plt.subplot(2, 1, 1)
        plt.plot(y, 'b-', alpha=0.5, label='Returns')
        plt.title(title)
        plt.legend()
        
        # Plot volatility
        plt.subplot(2, 1, 2)
        plt.plot(np.sqrt(self.fitted_volatility), 'r-', label='Estimated Volatility')
        
        if true_vol is not None:
            plt.plot(np.sqrt(true_vol), 'b-', alpha=0.3, label='True Volatility')
        
        plt.legend()
        plt.ylim(0, np.max(np.sqrt(self.fitted_volatility)) * 1.5)
        plt.tight_layout()
        plt.show()
    def simulate(self, T: int, dist: str = 't', df: int = 5, seed: int = None) -> Tuple[np.ndarray, np.ndarray]:
    
        if self.params is None:
            raise ValueError("Parameters must be set before simulation")

        if seed is not None:
            np.random.seed(seed)

        omega = self.params['omega']
        gamma = self.params['gamma']
        beta = self.params['beta']

        if self.alpha_loss is None:
            alpha_loss = self.params['alpha_loss']
        else:
            alpha_loss = self.alpha_loss

        if self.c is None:
            c = self.params['c']
        else:
            c = self.c

        # Initialize arrays
        y = np.zeros(T)
        f = np.zeros(T+1)
        f[0] = omega / (1 - beta)  # Start at unconditional variance

        # Generate innovations
        if dist == 't':
            eps = np.random.standard_t(df, T)
        else:
            eps = np.random.normal(0, 1, T)
            
        # Generate data
        for t in range(T):
            # Generate return
            y[t] = np.sqrt(f[t]) * eps[t]
            
            # Compute the score
            e_t = (y[t]**2 - f[t])
            psi_t = self._rho_derivative(e_t, alpha_loss, c) 
            
            # Update volatility
            f[t+1] = omega + gamma * psi_t + beta * f[t]
            #f[t+1] = max(f[t+1], 1e-12)  # Ensure positive volatility
            
        return y, f[1:]
import numpy as np



In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt
from typing import Callable, Dict, Tuple, Optional, Union
import autograd 
from autograd import grad, jacobian
import warnings
warnings.filterwarnings('ignore')



def demo_with_outliers():
    """Demonstrate the model's performance with outliers in the data"""
    # Set parameters for the data generating process
    true_params = {
        'omega': 0.07,
        'gamma': 0.11,
        'beta': 0.80,
        'alpha_loss': 1,
        'c': 1.2
    }
    
    # Create a model with the true parameters for simulation
    sim_model = RobustQLEVolatilityModel(alpha_loss=true_params['alpha_loss'])
    sim_model.params = true_params
    
    # Simulate clean data
    T = 6000
    y_clean, true_vol = sim_model.simulate(T, dist='n', df=7, seed=42)
    
    # Create a copy of the data with outliers
    y_outliers = y_clean.copy()
    
    # Add outliers at different positions and with different magnitudes
    
    # 1. Add a few isolated very large outliers
    np.random.seed(123)
    outlier_positions = np.random.choice(range(500, T-500), 10, replace=False)
    outlier_signs = np.random.choice([-1, 1], 10)
    
    for i, pos in enumerate(outlier_positions):
        # Create outliers that are 10-15 standard deviations from the mean
        local_std = np.sqrt(true_vol[pos])
        y_outliers[pos] = outlier_signs[i] * np.random.uniform(10, 15) * local_std
    
    # 2. Add a cluster of moderately large outliers (simulating a volatility burst)
    cluster_start = 3000
    for i in range(30):
        local_std = np.sqrt(true_vol[cluster_start + i])
        if i % 2 == 0:  # every other observation is an outlier
            y_outliers[cluster_start + i] = np.random.choice([-1, 1]) * np.random.uniform(5, 8) * local_std
    
    # Create models for comparison
    # L2 loss model (standard QLE, not robust)
    model_l2 = RobustQLEVolatilityModel(alpha_loss=2)
    
    # Cauchy loss model (should be robust)
    model_cauchy = RobustQLEVolatilityModel(alpha_loss=0)
    
    # Alpha=1 model (should be robust)
    model_alpha1 = RobustQLEVolatilityModel(alpha_loss=1)
    
    # Model with estimated alpha
    model_est = RobustQLEVolatilityModel()
    
    # Fit the models to the data with outliers
    print("\nFitting standard QLE model (alpha=2):")
    params_l2 = model_l2.fit(y_outliers, method='basin_hopping', niter=100, stepsize=0.1)
    
    print("\nFitting robust model with Cauchy loss (alpha=0):")
    params_cauchy = model_cauchy.fit(y_outliers,method='basin_hopping', niter=100, stepsize=0.1)
    
    print("\nFitting robust model with alpha=1:")
    params_alpha1 = model_alpha1.fit(y_outliers, method='basin_hopping', niter=100, stepsize=0.1)
    
    print("\nFitting model with estimated alpha:")
    params_est = model_est.fit(y_outliers, method='basin_hopping', niter=100, stepsize=0.1)
    
    # Compare volatility estimates
    fig, axes = plt.subplots(4, 1, figsize=(14, 16), sharex=True)
    
    # Plot returns with outliers
    axes[0].plot(y_outliers, 'b-', alpha=0.5)
    axes[0].set_title('Returns with Outliers')
    axes[0].set_ylabel('Returns')
    
    # Highlight outlier positions
    for pos in outlier_positions:
        axes[0].axvline(x=pos, color='r', linestyle='--', alpha=0.3)
    axes[0].axvspan(cluster_start, cluster_start + 30, color='r', alpha=0.1)
    
    # Plot estimated volatilities
    axes[1].plot(np.sqrt(model_l2.fitted_volatility), 'r-', label='L2 Loss (α=2)')
    axes[1].plot(np.sqrt(true_vol), 'k--', alpha=0.5, label='True Volatility')
    axes[1].set_title('L2 Loss (Standard QLE)')
    axes[1].set_ylabel('Volatility')
    axes[1].legend()
    
    axes[2].plot(np.sqrt(model_cauchy.fitted_volatility), 'g-', label='Cauchy Loss (α=0)')
    axes[2].plot(np.sqrt(true_vol), 'k--', alpha=0.5, label='True Volatility')
    axes[2].set_title('Cauchy Loss (Robust QLE)')
    axes[2].set_ylabel('Volatility')
    axes[2].legend()
    
    axes[3].plot(np.sqrt(model_alpha1.fitted_volatility), 'b-', label='α=1')
    axes[3].plot(np.sqrt(model_est.fitted_volatility), 'm-', label=f'Est. α={params_est["alpha_loss"]:.2f}')
    axes[3].plot(np.sqrt(true_vol), 'k--', alpha=0.5, label='True Volatility')
    axes[3].set_title('Comparison of Different Alpha Values')
    axes[3].set_ylabel('Volatility')
    axes[3].set_xlabel('Time')
    axes[3].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Zoom in on outlier cluster region
    fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    
    window_start = cluster_start - 100
    window_end = cluster_start + 150
    
    # Plot returns with outliers (zoomed)
    axes[0].plot(range(window_start, window_end), y_outliers[window_start:window_end], 'b-', alpha=0.7)
    axes[0].set_title('Returns with Outliers (Zoomed)')
    axes[0].set_ylabel('Returns')
    
    # Plot estimated volatilities (zoomed)
    axes[1].plot(range(window_start, window_end), np.sqrt(model_l2.fitted_volatility[window_start:window_end]), 
                 'r-', label='L2 Loss (α=2)')
    axes[1].plot(range(window_start, window_end), np.sqrt(model_cauchy.fitted_volatility[window_start:window_end]), 
                 'g-', label='Cauchy Loss (α=0)')
    axes[1].plot(range(window_start, window_end), np.sqrt(model_alpha1.fitted_volatility[window_start:window_end]), 
                 'b-', label='α=1')
    axes[1].plot(range(window_start, window_end), np.sqrt(model_est.fitted_volatility[window_start:window_end]), 
                 'm-', label=f'Est. α={params_est["alpha_loss"]:.2f}')
    axes[1].plot(range(window_start, window_end), np.sqrt(true_vol[window_start:window_end]), 
                 'k--', alpha=0.5, label='True Volatility')
    axes[1].set_title('Volatility Estimates Near Outlier Cluster')
    axes[1].set_ylabel('Volatility')
    axes[1].set_xlabel('Time')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

    # zoom in on isolated outliers
    fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    window_start = outlier_positions[0] - 100
    window_end = outlier_positions[0] + 100
    # Plot returns with outliers (zoomed)
    axes[0].plot(range(window_start, window_end), y_outliers[window_start:window_end], 'b-', alpha=0.7)
    axes[0].set_title('Returns with Outliers (Zoomed)')
    axes[0].set_ylabel('Returns')
    # Highlight outlier positions
    for pos in outlier_positions:
        axes[0].axvline(x=pos, color='r', linestyle='--', alpha=0.3)
    # Plot estimated volatilities (zoomed)
    axes[1].plot(range(window_start, window_end), np.sqrt(model_l2.fitted_volatility[window_start:window_end]),
                 'r-', label='L2 Loss (α=2)')
    axes[1].plot(range(window_start, window_end), np.sqrt(model_cauchy.fitted_volatility[window_start:window_end]),
                 'g-', label='Cauchy Loss (α=0)')
    axes[1].plot(range(window_start, window_end), np.sqrt(model_alpha1.fitted_volatility[window_start:window_end]),
                 'b-', label='α=1')
    axes[1].plot(range(window_start, window_end), np.sqrt(model_est.fitted_volatility[window_start:window_end]),
                 'm-', label=f'Est. α={params_est["alpha_loss"]:.2f}')
    axes[1].plot(range(window_start, window_end), np.sqrt(true_vol[window_start:window_end]),
                 'k--', alpha=0.5, label='True Volatility')
    axes[1].set_title('Volatility Estimates Near Isolated Outliers')
    axes[1].set_ylabel('Volatility')
    axes[1].set_xlabel('Time')
    axes[1].legend()
    plt.tight_layout()
    plt.show()  
    
    # Calculate RMSE for each model
    rmse_l2 = np.sqrt(np.mean((model_l2.fitted_volatility - true_vol)**2))
    rmse_cauchy = np.sqrt(np.mean((model_cauchy.fitted_volatility - true_vol)**2))
    rmse_alpha1 = np.sqrt(np.mean((model_alpha1.fitted_volatility - true_vol)**2))
    rmse_est = np.sqrt(np.mean((model_est.fitted_volatility - true_vol)**2))
    
    print("\nRMSE Comparison:")
    print(f"L2 Loss (α=2): {rmse_l2:.6f}")
    print(f"Cauchy Loss (α=0): {rmse_cauchy:.6f}")
    print(f"Alpha=1: {rmse_alpha1:.6f}")
    print(f"Estimated Alpha (α={params_est['alpha_loss']:.2f}): {rmse_est:.6f}")
    
    # Calculate mean absolute error near outliers
    near_outliers = np.zeros(T, dtype=bool)
    for pos in outlier_positions:
        # Mark 5 observations before and after each outlier
        start = max(0, pos-5)
        end = min(T, pos+6)
        near_outliers[start:end] = True
    
    # Also mark the cluster region
    near_outliers[cluster_start:cluster_start+40] = True
    
    # Calculate MAE near outliers
    mae_l2_near = np.mean(np.abs(model_l2.fitted_volatility[near_outliers] - true_vol[near_outliers]))
    mae_cauchy_near = np.mean(np.abs(model_cauchy.fitted_volatility[near_outliers] - true_vol[near_outliers]))
    mae_alpha1_near = np.mean(np.abs(model_alpha1.fitted_volatility[near_outliers] - true_vol[near_outliers]))
    mae_est_near = np.mean(np.abs(model_est.fitted_volatility[near_outliers] - true_vol[near_outliers]))
    
    print("\nMAE Near Outliers:")
    print(f"L2 Loss (α=2): {mae_l2_near:.6f}")
    print(f"Cauchy Loss (α=0): {mae_cauchy_near:.6f}")
    print(f"Alpha=1: {mae_alpha1_near:.6f}")
    print(f"Estimated Alpha (α={params_est['alpha_loss']:.2f}): {mae_est_near:.6f}")
    
    return model_l2, model_cauchy, model_alpha1, model_est, y_outliers, true_vol

if __name__ == "__main__":
    model_l2, model_cauchy, model_alpha1, model_est, y_outliers, true_vol = demo_with_outliers()


Fitting standard QLE model (alpha=2):
Fitting model using basin_hopping method...
basinhopping step 0: f 2.03787e-05
basinhopping step 1: f 2.19051e-05 trial_f 2.19051e-05 accepted 1  lowest_f 2.03787e-05
basinhopping step 2: f 2.19051e-05 trial_f 1e+10 accepted 0  lowest_f 2.03787e-05
basinhopping step 3: f 2.45878e-05 trial_f 2.45878e-05 accepted 1  lowest_f 2.03787e-05
basinhopping step 4: f 2.45878e-05 trial_f 8.85855e-05 accepted 0  lowest_f 2.03787e-05
basinhopping step 5: f 1.01992e-05 trial_f 1.01992e-05 accepted 1  lowest_f 1.01992e-05
found new global minimum on step 5 with function value 1.01992e-05
basinhopping step 6: f 1.01992e-05 trial_f 8.31591e-05 accepted 0  lowest_f 1.01992e-05
basinhopping step 7: f 1.01992e-05 trial_f nan accepted 0  lowest_f 1.01992e-05
basinhopping step 8: f 0.00460309 trial_f 0.00460309 accepted 1  lowest_f 1.01992e-05
basinhopping step 9: f 0.00510591 trial_f 0.00510591 accepted 1  lowest_f 1.01992e-05
basinhopping step 10: f 0.00463718 trial_